In [61]:
import pandas as pd
import io
import requests
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

# Replace with the URL you copied
url = 'https://raw.githubusercontent.com/jvilchesf/Learning/main/Andrej_karpathy_videos/Makemore/names.txt'

response = requests.get(url)
words = response.text.splitlines()

In [62]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i ,s in enumerate(chars)} 
stoi['.']= 0
itos = {i:s for s,i in stoi.items()}

In [64]:
#Create the training set of biagrams
b = {}
xs = []
ys = []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        x,y = stoi[ch1], stoi[ch2]
        xs.append(x)
        ys.append(y)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

#Total of words
elements =  xs.nelement()
print(f"Total of words: {elements}")

#create a generator variable to repeat always the same random numbers
g = torch.Generator().manual_seed(2147483647)
#generate 27 random weights and 27 neurons (weigs x neurons = 27 x 27)  
W = torch.randn((27,27), requires_grad = True, generator = g)

Total of words: 228146


In [67]:
for i in range(100):
    #Forward pass
    #A neuronal network doesn't get the input as a number, it gets it as a one hot encoding format
    one_hot_xs = F.one_hot(xs, num_classes=27).float()
    #Create the logits, multiplying the one hot encoding by the weights
    logits = one_hot_xs @ W 
    #logits have to be > 0, so we apply the softmax function
    counts = logits.exp()
    #Normalization to create probabilities 
    probs = counts / counts.sum(1, keepdim = True)
    #Loss function
    loss = -probs[torch.arange(elements),ys].log().mean()

    #backward pass
    W.grad = None
    loss.backward()

    #Update weights
    W.data -= W.grad * 50

print(loss.item())

2.4611728191375732


In [71]:
#Test the model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):

    out = []
    ix = 0
    while True:

        exc = F.one_hot(torch.tensor([ix]), num_classes = 27).float()
        logits = exc @ W
        count = logits.exp()
        prob = count / count.sum(1, keepdim = True)

        ix = torch.multinomial(prob, num_samples = 1, generator = g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

junide.
janasah.
prelay.
a.
nn.
